In [37]:
import csv
import pprint
import codecs
import json

In [31]:
INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def write_data(file, data, header):
    with open(file, 'w') as f:
        writer = csv.DictWriter(f, delimiter=",", fieldnames=header)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        good_data = []
        bad_data = []
        for row in reader:
            # Only want data from dbpedia.org, do not put other data into either file
            if (row['URI'] == 'URI') or row['URI'] == ('http://www.w3.org/2002/07/owl#Thing'):
                pass
            else:
                production_start_year = row['productionStartYear'][0:4]
                try:
                    production_start_year = int(production_start_year)
                    if (1886 < production_start_year < 2014):
                        row['productionStartYear'] = production_start_year
                        good_data.append(row)
                    else:
                        bad_data.append(row)
                except:
                    bad_data.append(row)

    write_data(output_good, good_data, header)
    write_data(output_bad, bad_data, header)

process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)

In [33]:
exString = 'this is some text'

In [34]:
exString.find('text')

13

In [36]:
exString.find('ok')

-1

In [54]:
CITIES = 'cities.csv'

In [55]:
FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

In [73]:
def determine_type(data):
    if (data == 'NULL') or (data == ""):
        return type(None)
    elif data[0] == "{":
        return type([])
    else:
        try:
            float(data)
            try:
                int(data)
                return type(int())
            except:
                return type(float())
        except:
            return type(str())

def audit_file(filename, fields):
    
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        headers = reader.fieldnames
        # Create empty dictionary where each value is a set of data types
        # The keys of the dictionary are the fields
        for field in fields:
            type_dict[field] = set([])
            
        for row in reader:
            if row['URI'].find('dbpedia.org') < 0:
                    pass
            else:
                for field in fields:
                    type_dict[field].add(determine_type(row[field]))

    return type_dict

audit_file(CITIES,FIELDS)
            

{'areaCode': {int, str, NoneType},
 'areaLand': {list, NoneType, float},
 'areaMetro': {NoneType, float},
 'areaUrban': {NoneType, float},
 'elevation': {list, NoneType, float},
 'governmentType_label': {str, NoneType},
 'homepage': {str, NoneType},
 'isPartOf_label': {list, NoneType, str},
 'maximumElevation': {NoneType},
 'minimumElevation': {NoneType},
 'name': {str, NoneType, list},
 'populationDensity': {list, NoneType, float},
 'populationTotal': {int, NoneType},
 'timeZone_label': {str, NoneType},
 'utcOffset': {int, str, NoneType, list},
 'wgs84_pos#lat': {float},
 'wgs84_pos#long': {float}}

In [74]:
CITIES = 'cities.csv'

In [101]:
def fix_area(area):
    # Remove the braces from area
    area = area.strip("{}")
    
    if area == 'NULL':
        return None
    else:
        #split on the pipe character
        area = area.split("|")
        try:
            return float(area)
        except:
            return(float(max(area, key=len)))
                
def fix_name(name):
    # Remove the braces from name
    name= name.strip("{}")
    
    if name == "NULL":
        return []
    else:
        name = name.split("|")
        return name

def check_loc(point, lat, longi):
    point = point.split(" ")
    if (point[0] == lat) and (point[1]==longi):
        return True
    else:
        return False
    
    
def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        headers = reader.fieldnames
        #skipping the extra metadata
        for row in reader:
            if row['URI'].find('dbpedia.org') < 0:
                pass

        # processing file
            else:
                row['Point Correct'] = False
            # calling your function to fix the area value
                if "areaLand" in row:
                    
                    row["areaLand"] = fix_area(row['areaLand'])
                
                
                if "name" in row:
                    row['name'] = fix_name(row['name'])
                    
                if 'point' in row:
                    row['Point Correct'] == check_loc(row['point'], row["wgs84_pos#lat"], row["wgs84_pos#long"])
                    
                data.append(row)
    return data


cities_data = process_file(CITIES)